In [27]:
import re
q = "ERBB2, ERBB4, FGFR2, FGFR4, HRAS, KRAS are involved in RTK signaling, ABC signaling"
re.findall(r"[A-Za-z0-9_\-]+", q)

['ERBB2',
 'ERBB4',
 'FGFR2',
 'FGFR4',
 'HRAS',
 'KRAS',
 'are',
 'involved',
 'in',
 'RTK',
 'signaling',
 'ABC',
 'signaling']

In [25]:
from collections import defaultdict
aliases = defaultdict(set)
aliases['Rtk'.lower()].add('RTK signaling')
aliases['ABC'.lower()].add('ABC signaling')
aliases['signaling'.lower()].add('RTK signaling')
aliases['signaling'.lower()].add('ABC signaling')

found = set()
for token in re.findall(r"[A-Za-z0-9_\-]+", q):
    token_l = token.lower()
    print(token_l)
    if token_l in aliases:
        found.update(aliases[token_l])
found

erbb2
erbb4
fgfr2
fgfr4
hras
kras
are
involved
in
rtk
signaling


{'ABC signaling', 'RTK signaling'}

In [28]:
nodes = ["ERBB2", "ERBB4", "FGFR2", "FGFR4", "HRAS", "KRAS", "RTK signaling", "ABC signaling"]
for tok in re.findall(r"\b[A-Za-z0-9]{1,25}\b", q):
    print(tok)
    if tok in nodes:
        found.add(tok)
    if tok.lower() in aliases:
        found.update(aliases[tok.lower()])

ERBB2
ERBB4
FGFR2
FGFR4
HRAS
KRAS
are
involved
in
RTK
signaling
ABC
signaling


In [29]:
found

{'ABC signaling',
 'ERBB2',
 'ERBB4',
 'FGFR2',
 'FGFR4',
 'HRAS',
 'KRAS',
 'RTK signaling'}

In [16]:
aliases

defaultdict(set, {})

In [5]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="nebius",
    api_key=os.environ["HF_TOKEN"],
)

result = client.feature_extraction(
    "ERBB2, ERBB4, FGFR2, FGFR4, HRAS, KRAS are involved in RTK signaling",
    model="Qwen/Qwen3-Embedding-8B",
)

In [6]:
result.shape

(1, 4096)

In [7]:
result2 = client.feature_extraction(
    "RTK signaling",
    model="Qwen/Qwen3-Embedding-8B",
)

In [8]:
result2.shape

(1, 4096)

In [9]:
result1 = client.sentence_similarity(
    result2,
    result
)
result1

ValueError: Task 'sentence-similarity' not supported for provider 'nebius'. Available tasks: ['text-to-image', 'conversational', 'text-generation', 'feature-extraction']

In [2]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re
import logging
# Load model directly
model = SentenceTransformer("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
def extract_relevant_sentences_from_node(
                                         node_id,
                                             query_terms,
                                             model=None,
                                             top_n: int =3):
        
        node = ["ERBB2", "ERBB4", "FGFR2", "FGFR4", "HRAS", "KRAS", "RTK signaling", "ABC signaling"]
        text = """
        PMID: 18050474
        Title: RTK/Ras/MAPK signaling.
        Abstract: Receptor Tyrosine Kinase (RTK)/Ras GTPase/MAP kinase (MAPK) signaling pathways are used repeatedly during metazoan development to control many different biological processes. In the nematode Caenorhabditis elegans, two different RTKs (LET-23/EGFR and EGL-15/FGFR) are known to stimulate LET-60/Ras and a MAPK cascade consisting of the kinases LIN-45/Raf, MEK-2/MEK and MPK-1/ERK. This Ras/MAPK cascade is required for multiple developmental events, including induction of vulval, uterine, spicule, P12 and excretory duct cell fates, control of sex myoblast migration and axon guidance, and promotion of germline meiosis. Studies in C. elegans have provided much insight into the basic framework of this RTK/Ras/MAPK signaling pathway, its regulation, how it elicits cell-type specific responses, and how it interacts with other signaling pathways such as the Wnt and Notch pathways.
        PMID: 28589435
        Title: Epigenetic regulation of RTK signaling.
        Abstract: Receptor tyrosine kinase (RTK) signaling cascades coordinate intracellular signaling in response to growth factors, chemokines, and other extracellular stimuli to control fundamental biological processes such as cellular proliferation, metabolism, and survival. Hyperactivation of pathways associated with growth factor signaling (e.g., RTK and downstream effectors including Ras, PI3K/AKT, and Raf) is a frequent event in human cancers, which uncouples ligand-mediated activation with signal transduction. While the contributions of direct genomic events are well understood as causative agents of hyperactive signal transduction, other non-heritable genomic modifications promote the activation of growth factor-associated signaling cascades. In this review, we highlight epigenomic mechanisms by which hyperactivation of RTK-associated signaling cascades occurs and may contribute to cancer.
        PMID: 33346130
        Title: Relationships between DNA repair and RTK-mediated signaling pathways.
        Abstract: Receptor Tyrosine Kinases (RTK) are an important family involved in numerous signaling pathways essential for proliferation, cell survival, transcription or cell-cycle regulation. Their role and involvement in cancer cell survival have been widely described in the literature, and are generally associated with overexpression and/or excessive activity in the cancer pathology. Because of these characteristics, RTKs are relevant targets in the fight against cancer. In the last decade, increasingly numerous works describe the role of RTK signaling in the modulation of DNA repair, thus providing evidence of the relationship between RTKs and the protein actors in the repair pathways. In this review, we propose a summary of RTKs described as potential modulators of double-stranded DNA repair pathways in order to put forward new lines of research aimed at the implementation of new therapeutic strategies targeting both DNA repair pathways and RTK-mediated signaling pathways.
        PMID: 39915447
        Title: Hepatocellular carcinoma: signaling pathways and therapeutic advances.
        Abstract: Liver cancer represents a major global health concern, with projections indicating that the number of new cases could surpass 1 million annually by 2025. Hepatocellular carcinoma (HCC) constitutes around 90% of liver cancer cases and is primarily linked to factors incluidng aflatoxin, hepatitis B (HBV) and C (HCV), and metabolic disorders. There are no obvious symptoms in the early stage of HCC, which often leads to delays in diagnosis. Therefore, HCC patients usually present with tumors in advanced and incurable stages. Several signaling pathways are dis-regulated in HCC and cause uncontrolled cell propagation, metastasis, and recurrence of HCC. Beyond the frequently altered and therapeutically targeted receptor tyrosine kinase (RTK) pathways in HCC, pathways involved in cell differentiation, telomere regulation, epigenetic modification and stress response also provide therapeutic potential. Investigating the key signaling pathways and their inhibitors is pivotal for achieving therapeutic advancements in the management of HCC. At present, the primary therapeutic approaches for advanced HCC are tyrosine kinase inhibitors (TKI), immune checkpoint inhibitors (ICI), and combination regimens. New trials are investigating combination therapies involving ICIs and TKIs or anti-VEGF (endothelial growth factor) therapies, as well as combinations of two immunotherapy regimens. The outcomes of these trials are expected to revolutionize HCC management across all stages. Here, we provide here a comprehensive review of cellular signaling pathways, their therapeutic potential, evidence derived from late-stage clinical trials in HCC and discuss the concepts underlying earlier clinical trials, biomarker identification, and the development of more effective therapeutics for HCC.
        PMID: 30026122
        Title: A quantitative model of developmental RTK signaling.
        Abstract: Receptor tyrosine kinases (RTKs) control a wide range of developmental processes, from the first stages of embryogenesis to postnatal growth and neurocognitive development in the adult. A significant share of our knowledge about RTKs comes from genetic screens in model organisms, which provided numerous examples demonstrating how specific cell fates and morphologies are abolished when RTK activation is either abrogated or significantly reduced. Aberrant activation of such pathways has also been recognized in many forms of cancer. More recently, studies of human developmental syndromes established that excessive activation of RTKs and their downstream signaling effectors, most notably the Ras signaling pathway, can also lead to structural and functional defects. Given that both insufficient and excessive pathway activation can lead to abnormalities, mechanistic analysis of developmental RTK signaling must address quantitative questions about its regulation and function. Patterning events controlled by the RTK Torso in the early Drosophila embryo are well-suited for this purpose. This mini review summarizes current state of knowledge about Torso-dependent Ras activation and discusses its potential to serve as a quantitative model for studying the general principles of Ras signaling in development and disease.
            """
        # if not text:
        #     return []

        sentences = [s.strip() for s in re.split(r'(?<=[\.\?\!])\s+', text) if s.strip()]

        # q_terms_low = [t.lower() for t in query_terms]

        # # 1) direct substring matches -> score 1.0
        # hits = []
        # for s in sentences:
        #     s_low = s.lower()
        #     if any(q in s_low for q in q_terms_low):
        #         hits.append((1.0, s))

        # if hits:
        #     # return top_n substring hits (preserve order)
        #     return hits[:top_n]

        # 2) fallback: if model available, use embedding similarity
        if model is not None:
            # encode query and sentences
            try:
                # we build a query by joining the terms
                q_text = " ".join(query_terms)
                emb_q = model.encode([q_text], convert_to_numpy=True, show_progress_bar=False)
                emb_sent = model.encode(sentences, convert_to_numpy=True, show_progress_bar=False)
                # normalize
                emb_q = emb_q / np.linalg.norm(emb_q, axis=1, keepdims=True)
                emb_sent = emb_sent / np.linalg.norm(emb_sent, axis=1, keepdims=True)
                sims = (emb_sent @ emb_q.T).squeeze()  # cosine via inner product
                scored = sorted([(float(sims[i]), sentences[i]) for i in range(len(sentences))], reverse=True)
                return scored[:top_n]
            except Exception:
                # if embedding fails, return first N sentences as last resort
                return [(0.0, s) for s in sentences[:top_n]]

        # 3) final fallback: return first N sentences, score 0
        return [(0.0, s) for s in sentences[:top_n]]

No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubMedBERT-fulltext. Creating a new one with mean pooling.


In [6]:
q = ["ERBB2", "ERBB4", "FGFR2", "FGFR4", "HRAS", "KRAS", "RTK signaling"]
l = extract_relevant_sentences_from_node(node_id="ABCD", query_terms=q, model=model, top_n=10)

In [7]:
for score, sent in l:
    print(f"{sent}")

Hyperactivation of pathways associated with growth factor signaling (e.g., RTK and downstream effectors including Ras, PI3K/AKT, and Raf) is a frequent event in human cancers, which uncouples ligand-mediated activation with signal transduction.
PMID: 18050474
        Title: RTK/Ras/MAPK signaling.
Abstract: Receptor Tyrosine Kinase (RTK)/Ras GTPase/MAP kinase (MAPK) signaling pathways are used repeatedly during metazoan development to control many different biological processes.
Abstract: Receptor Tyrosine Kinases (RTK) are an important family involved in numerous signaling pathways essential for proliferation, cell survival, transcription or cell-cycle regulation.
Abstract: Receptor tyrosine kinase (RTK) signaling cascades coordinate intracellular signaling in response to growth factors, chemokines, and other extracellular stimuli to control fundamental biological processes such as cellular proliferation, metabolism, and survival.
In this review, we highlight epigenomic mechanisms by 

In [8]:
l

[(0.7139331102371216,
  'Hyperactivation of pathways associated with growth factor signaling (e.g., RTK and downstream effectors including Ras, PI3K/AKT, and Raf) is a frequent event in human cancers, which uncouples ligand-mediated activation with signal transduction.'),
 (0.6061835289001465,
  'PMID: 18050474\n        Title: RTK/Ras/MAPK signaling.'),
 (0.5957146883010864,
  'Abstract: Receptor Tyrosine Kinase (RTK)/Ras GTPase/MAP kinase (MAPK) signaling pathways are used repeatedly during metazoan development to control many different biological processes.'),
 (0.5951964855194092,
  'Abstract: Receptor Tyrosine Kinases (RTK) are an important family involved in numerous signaling pathways essential for proliferation, cell survival, transcription or cell-cycle regulation.'),
 (0.5884079337120056,
  'Abstract: Receptor tyrosine kinase (RTK) signaling cascades coordinate intracellular signaling in response to growth factors, chemokines, and other extracellular stimuli to control fundame